In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks
from scipy.fftpack import ifft2
import pandas as pd
import time
# import websocket
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# import mouse


In [2]:
def startDriver():
    driver = webdriver.Chrome(executable_path = r"C:\Users\Webdriver\chromedriver.exe")
    return driver

def getToScreener(driver):
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/index.html'
    driver.get(URL)
    return driver

def getToDarkness(driver):
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/black.html'
    driver.get(URL)
    return driver

def clickNumber(number, driver):
    button = driver.find_element(By.ID, f'key{number}')
    button.click()

def on_message(ws, message):
    print(message)

def power_spectrum(fft):
    # Returns the power spectrum of the 2D-Fourier Transform
    return np.abs(fft)**2

def FFT(image):
    # Return the fast fourier transform after it is centered
    fft = np.fft.fft2(image, s=None, axes=(-2, -1), norm=None)
    fft = np.fft.fftshift(fft, axes=None)
    return fft

def inverse_FFT(pwr_spectrum):
    # Compute the inverse Fourier transform of the power spectrum
    fft_array = np.fft.ifftshift(pwr_spectrum, axes=None)
    img = ifft2(fft_array)

    # Take the real part of the image to remove any imaginary components
    img = np.real(img)

    # Normalize the pixel values to the range [0, 255]
    img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))

    # Convert the pixel values to integers and return the image
    return np.abs(img.astype(np.uint8)) ** 2

def maskDots(fft_img, X, Y, radius_squared):
    # Compute the squared distances from the circle center
    r1 = (np.arange(crop_size)[:, None] - Y - crop_size/2) ** 2 + (np.arange(crop_size) - X - crop_size/2) ** 2
    r2 = (np.arange(crop_size)[:, None] + Y - crop_size/2) ** 2 + (np.arange(crop_size) + X - crop_size/2) ** 2

    
    # Create a mask that is True where both distances are greater than radius ** 2
    mask = np.logical_and(r1 > radius_squared, r2 > radius_squared)

    # Apply the mask to the FFT image
    fft_img[mask] = 0
    
    fft_copy = fft_img.copy()

    return fft_copy

def maskLine(fft_img, X, Y, radius_squared):
    r = []
    for i in range(10):
        r.append((np.arange(crop_size)[:, None] - Y + (2*Y/10*i) - crop_size/2) ** 2 + (np.arange(crop_size) - X + (2*X/10*i) - crop_size/2) ** 2)
        
    center = (np.arange(crop_size)[:, None] - crop_size/2) ** 2 + (np.arange(crop_size) - crop_size/2) ** 2
    # Create a mask that is True where both distances are greater than radius ** 2
    
    mask = r[0] > radius_squared
    
    for i in range(9):
        if i not in [3,4,5,6]:
            mask = np.logical_and(mask, r[i+1] > radius_squared)

    # Apply the mask to the FFT image
    fft_img[mask] = 0
    
    fft_copy = fft_img.copy()

    return fft_copy

def maskSquare(a, radius, X, Y):
    X = int(X + crop_size/2)
    Y = int(Y + crop_size/2)
    b = np.array(a.copy(), dtype=np.uint8)
    return b[X - radius:X + radius,Y - radius:Y + radius]

def mask(fft_img, height, width, X, Y, radius_squared):
    return maskDots(fft_img, height, width, X, Y, radius_squared)

def planeInteraction(peak):
    return fft_dist[int(peak[0]+crop_size/2)][int(peak[1]+crop_size/2)] > 200

def keyAreaPressed(img):
    # Takes image returns which key is pressed
    keys, points = 3, 10
    height, width = len(img)/(keys), len(img[0])/(keys) # Determines the vertical and horizontal distance moved
        
    total = []
    for h in range(keys-1,-1,-1):
        for w in range(keys):
            value = 0
            for i in range(points):
                for j in range(points):
                    # Adds the total pixel value of all points close to center of key
                    value += img[int(h*height + height/4 + height/2*(i+1)/points)][int(w*width + width/4 + width/2*(i+1)/points)]
            total.append(value)
    ID = f'key{total.index(max(total))+1}'
    return ID # Returns the key that has the highest total

def lowToHigh(currentB, previousB):
    passed = False
    if currentB and not previousB:
        passed = True
    return passed

# Define the distortion coefficients
k1 = -0.8  # pincushion distortion
k2 = 0.0
k3 = 0.0
p1 = 0.0
p2 = 0.0
dist_coeffs = np.array([k1, k2, k3, p1, p2], dtype=np.float32)
    
def distortImage(img):
    # Distortion values
    
    # Define the camera matrix
    focal_length = 300
    center_x = img.shape[1] / 2
    center_y = img.shape[0] / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)
    
    # Apply pincushion distortion to the image
    img_distorted = cv2.undistort(img, camera_matrix, dist_coeffs)
    
    return img_distorted

def findPeakGrid(a):
    size = len(a)
    b = np.asarray(a)
    
    row = 0
    column = b.argmax()
    while column >= size:
        column -= size
        row += 1
    
    return [column,row]

def xyPeak(a):
    peak = findPeakGrid(a)
    return [peak[0]+X-radius,peak[1]+Y-radius]

def cameraInDarkness(cap):
    cap.set(cv2.CAP_PROP_EXPOSURE, -2) 
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 104) 
    cap.set(cv2.CAP_PROP_SATURATION, 0) 
    cap.set(cv2.CAP_PROP_CONTRAST, 255) 
    cap.set(cv2.CAP_PROP_SHARPNESS, 165) 
    cap.set(cv2.CAP_PROP_FOCUS, 18) 
    cap.set(cv2.CAP_PROP_GAIN, 255) 
    cap.set(cv2.CAP_PROP_TILT, 0) 
    cap.set(cv2.CAP_PROP_PAN, 0) 
    cap.set(cv2.CAP_PROP_ZOOM, 390) 
    
def cameraInLight(cap):
    cap.set(cv2.CAP_PROP_EXPOSURE, -2) 
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 230) 
    cap.set(cv2.CAP_PROP_CONTRAST, 255) 
    cap.set(cv2.CAP_PROP_SHARPNESS, 130) 
    cap.set(cv2.CAP_PROP_FOCUS, 18) 
    cap.set(cv2.CAP_PROP_GAIN, 60) 
    cap.set(cv2.CAP_PROP_TILT, -1) 
    cap.set(cv2.CAP_PROP_PAN, 0) 
    cap.set(cv2.CAP_PROP_ZOOM, 500) 
    return cap
    
def getComputerCamera():
    return cv2.VideoCapture(0)
    
def getWebCam():
    return cv2.VideoCapture(1 + cv2.CAP_DSHOW)

def changeCameraSetting(cap):
    cap.set(cv2.CAP_PROP_SETTINGS, 1) # Set settings
    return cap

In [3]:
# Mask values to be edited
# Y, X, radius_squared = -20, 135, 1000
Y, X, radius_squared, radius = 10, 16, 40, 4
crop_size = 300

In [4]:
# Start Camera
cap = getWebCam()
cameraInDarkness(cap)
changeCameraSetting(cap)

< cv2.VideoCapture 00000203212B0770>

In [5]:
# Open the keyboard using selenium
driver = startDriver()
keyboard = getToDarkness(driver)

previousClicked = False
firstRun = True
firstImageCaptured = False

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = frame[0:crop_size, 0:crop_size]

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    while firstRun:
        first_gray = gray
        firstRun = False
    
    # Capture background frame when user presses 'f'
    if cv2.waitKey(1) & 0xFF == ord('f'):
        first_gray = gray
        keyboard = getToScreener(driver)
        firstImageCaptured = True
        
    img1 = np.concatenate((gray,power_spectrum(FFT(gray))),axis=0)

    # Difference in images
    diff_img = cv2.absdiff(gray, first_gray)
    dist_img = distortImage(gray)
    
    img2 = np.concatenate((dist_img,power_spectrum(FFT(dist_img))),axis=0)
    
    fft_dist = FFT(dist_img)
    only_mask = maskSquare(fft_dist, radius, X, Y)
    peak = xyPeak(only_mask)
    
    masked_img = maskDots(fft_dist, peak[0], peak[1], radius_squared)
    final_img = inverse_FFT(masked_img)
    
    img3 = np.concatenate((final_img,power_spectrum(masked_img)),axis=0)
    
    img_total = np.concatenate((img1,img2,img3),axis=1)
    
    cv2.imshow('allWindows', img_total)
    cv2.imshow("gray",gray)
    
    
    # Tell if plane is interacted with and which key is pressed
    if firstImageCaptured:
        currentClicked = planeInteraction(peak)
        if currentClicked:
            #print((keyAreaPressed(final_img)), end="\r")
            if lowToHigh(currentClicked, previousClicked):
                keyboard.find_element(By.ID, keyAreaPressed(final_img)).click()
                #print(currentClicked, previousClicked)
        previousClicked = currentClicked

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()
keyboard.close()

C:\Users\Bruger\AppData\Local\Temp\ipykernel_10148\624671286.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r"C:\Users\Webdriver\chromedriver.exe")
C:\Users\Bruger\AppData\Local\Temp\ipykernel_10148\624671286.py:86: ComplexWarning: Casting complex values to real discards the imaginary part
  b = np.array(a.copy(), dtype=np.uint8)
C:\Users\Bruger\AppData\Local\Temp\ipykernel_10148\624671286.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))
C:\Users\Bruger\AppData\Local\Temp\ipykernel_10148\624671286.py:41: RuntimeWarning: invalid value encountered in multiply
  img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))
